

We create an instance of a skyjo_env environment, call reset() to initialize the game and list the available agents (players):


In [ ]:
import numpy as np

from rlskyjo.environment import skyjo_env
from importlib import reload

reload(skyjo_env)

<module 'rlskyjo.environment.skyjo_env' from '/home/michi/skybo_rl/rlskyjo/environment/skyjo_env.py'>

In [2]:
skyjo_env_cfg = {"num_players": 3}
env_pettingzoo = skyjo_env.env(**skyjo_env_cfg)
env_pettingzoo.reset()

env_pettingzoo.agents, env_pettingzoo.agent_selection


def sample_place():
    return np.random.randint(0, 23)


def sample_draw():
    return np.random.randint(24, 25)

In [3]:
print(env_pettingzoo.agent_selection, "draw")
env_pettingzoo.step(sample_draw())
print(env_pettingzoo.agent_selection, "place")
env_pettingzoo.step(sample_place())

player_0 draw
player_0 place
[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.


In [4]:
def random_admissible_policy(observation, action_mask):
    """picks randomly an admissible action from the action mask"""
    return np.random.choice(
        np.arange(len(action_mask)), p=action_mask / np.sum(action_mask)
    )


assert 1 not in [random_admissible_policy(None, [1, 0, 1]) for _ in range(1000)]

In [5]:
i_episode = 1
while i_episode <= 1:
    i_episode += 1
    env_pettingzoo.reset()
    for agent in env_pettingzoo.agent_iter(max_iter=600):
        # get observation (state) for current agent:
        obs, reward, done, info = env_pettingzoo.last()

        print("training fct:", obs, reward, done, info)
        # perform q-learning with update_Q_value()
        # your code here

        env_pettingzoo.render()

        # store current state
        if not done:
            # choose action using epsilon_greedy_policy()
            # your code here
            observation = obs["observations"]
            action_mask = obs["action_mask"]
            action = random_admissible_policy(observation, action_mask)

            print(f"sampled action {agent}: {action}")
            env_pettingzoo.step(action)
        else:
            # agent is done
            env_pettingzoo.step(None)
            print("done", reward)
            break


else:
    print(env_pettingzoo._cumulative_rewards)

training fct: {'observations': array([ 5, 10,  0,  0,  1,  0,  0,  1,  0,  1,  0,  0,  3,  0,  0,  0,  1,
        8, 15, 15, 12, 15, 15, 15, 15, 15, 15,  8, 15, 15, 15,  8, 15, 15,
       15, 15, 15,  3, 15, 15, 15, 15, 15, 15, 15,  0, 15,  5, 15, 15, 15,
       15, 15, 15, 15], dtype=int8), 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1], dtype=int8)} 0 False {}
======= render board: ===== 
======= stats ============ 
next turn: draw by Player 0 
holding card player 0: empty 
discard pile top: 8 
======= Player 0 ========== 
[[u	 u	 u	 u]
 [12	 u	 u	 u]
 [u	 u	 8	 u]]
======= Player 1 ========== 
[[8	 u	 3	 u]
 [u	 u	 u	 u]
 [u	 u	 u	 u]]
======= Player 2 ========== 
[[u	 u	 u	 u]
 [u	 5	 u	 u]
 [0	 u	 u	 u]]

sampled action player_0: 25
training fct: {'observations': array([ 5, 10,  0,  0,  1,  0,  0,  1,  0,  1,  0,  0,  2,  0,  0,  0,  1,
       -3,  8, 15, 12, 15, 15, 15, 15, 15, 15,  8, 15, 15, 15,  8, 15, 15,
       15, 

more envs test with rllib

In [6]:
from ray import init

init(num_cpus=os.cpu_count())

{'node_ip_address': '172.22.125.215',
 'raylet_ip_address': '172.22.125.215',
 'redis_address': '172.22.125.215:6379',
 'object_store_address': '/tmp/ray/session_2022-01-30_00-11-54_673538_18806/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-30_00-11-54_673538_18806/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-01-30_00-11-54_673538_18806',
 'metrics_export_port': 60361,
 'node_id': '34cffa258ec88ac704e62ccedb5526415bf6d1452b8ff4614d35e31a'}

In [7]:
from ray.tune.registry import register_env
from rlskyjo.environment import skyjo_env
from ray.rllib.env import PettingZooEnv
import ray.rllib.agents.dqn as dqn
from ray.rllib.agents import ppo
from copy import deepcopy
from ray.rllib.agents.registry import get_agent_class
from ray.rllib.models import ModelCatalog
from rlskyjo.models.action_mask_model import TorchMaskedActions, TorchActionMaskModel
from ray.rllib.models import ModelCatalog
from ray.rllib.agents.dqn import DQNTrainer, DQNTorchPolicy
from ray.tune.logger import pretty_print
import os
import torch

In [8]:
env_name = "pettingzoo_skyjo"


def env_creator():
    env = skyjo_env.env(**skyjo_env_cfg)
    return env


register_env(env_name, lambda config: PettingZooEnv(env_creator()))
ModelCatalog.register_custom_model("pa_model2", TorchActionMaskModel)
env = PettingZooEnv(env_creator())

/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")


In [9]:
env.env.observe(env.env.agent_selection)

{'observations': array([ 4, 10,  0,  1,  0,  1,  0,  1,  0,  2,  0,  1,  0,  1,  0,  0,  0,
         7, 15, 15, 15, 15, -1, 15, 15,  5, 15, 15, 15, 15, 15, 15, 15,  9,
        15, 15, 15,  1, 15, 15, 15, 15, 15, 15, 15, 15, 15,  5,  3, 15, 15,
        15, 15, 15, 15], dtype=int8),
 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1], dtype=int8)}

with multiagent

In [10]:
custom_config = {
    "env": env_name,
    "model": {
        "custom_model": "pa_model2",
    },
    "framework": "torch",
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": int(torch.cuda.device_count()),
    "num_workers": os.cpu_count() - 1,
    "multiagent": {
        "policies": {
            name: (None, env.observation_space, env.action_space, {})
            for name in env.agents
        },
        "policy_mapping_fn": lambda agent_id: agent_id,
    },
}


ppo_config = ppo.DEFAULT_CONFIG.copy()
ppo_config.update(custom_config)

trainer = ppo.PPOTrainer(config=ppo_config)

2022-01-30 00:11:59,495	WARNING ppo.py:143 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=15 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 266.
2022-01-30 00:11:59,496	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-01-30 00:11:59,496	INFO trainer.py:743 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=18989) /home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
(RolloutWorker pid=18989)   warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
(RolloutWorker pid=18989) /hom

In [11]:
# run manual training loop and print results after each iteration
max_steps = 2e6
max_iters = 100
for iters in range(max_iters):
    result = trainer.train()
    if iters % 2 == 0:
        print(pretty_print(result))
    # stop training if the target train steps or reward are reached
    if result["timesteps_total"] >= max_steps:
        print(
            f"training done, because max_steps {max_steps} {result['timesteps_total']} reached"
        )
        break
else:
    print(f"training done, because max_iters {max_iters} reached")
# manual test loop
print("Finished training. Running manual test/inference loop.")
# prepare environment with max 10 steps

(RolloutWorker pid=18989) 2022-01-30 00:12:19,619	WARNING deprecation.py:45 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!
(RolloutWorker pid=18989) /home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
(RolloutWorker pid=18989)   ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
(RolloutWorker pid=18989) /home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
(RolloutWorker pid=18989)   ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


agent_timesteps_total: 7950
custom_metrics: {}
date: 2022-01-30_00-12-42
done: false
episode_len_mean: 106.72727272727273
episode_media: {}
episode_reward_max: 3.000000000000014
episode_reward_mean: 3.0000000000000036
episode_reward_min: 2.999999999999986
episodes_this_iter: 66
episodes_total: 66
experiment_id: 736c238500184c1faf884c2b6581ce08
hostname: lenovo-michi
info:
  learner:
    player_0:
      learner_stats:
        cur_kl_coeff: 0.2
        cur_lr: 5.000000000000001e-05
        entropy: 1.7204936081171036
        entropy_coeff: 0.0
        kl: 0.011820796451732007
        policy_loss: -0.06959378906525672
        total_loss: 614.5029327678681
        vf_explained_var: 0.008369981348514556
        vf_loss: 614.5701631991069
    player_1:
      learner_stats:
        cur_kl_coeff: 0.2
        cur_lr: 5.000000000000001e-05
        entropy: 1.7231340459982554
        entropy_coeff: 0.0
        kl: 0.010285974530027574
        policy_loss: -0.03197940264518062
        total_loss: 

E0130 00:29:40.956252107   30179 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643498980.956227260","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643498980.956223583","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}


agent_timesteps_total: 454830
custom_metrics: {}
date: 2022-01-30_00-29-56
done: false
episode_len_mean: 313.96
episode_media: {}
episode_reward_max: 3.000000000000014
episode_reward_mean: 3.0
episode_reward_min: 2.999999999999986
episodes_this_iter: 26
episodes_total: 2585
experiment_id: 736c238500184c1faf884c2b6581ce08
hostname: lenovo-michi
info:
  learner:
    player_0:
      learner_stats:
        cur_kl_coeff: 0.45
        cur_lr: 5.000000000000001e-05
        entropy: 0.8341989959279696
        entropy_coeff: 0.0
        kl: 0.015439362842436709
        policy_loss: -0.06870315346090744
        total_loss: 101.0763548374176
        vf_explained_var: 0.14213007052739463
        vf_loss: 101.138110871315
    player_1:
      learner_stats:
        cur_kl_coeff: 0.3
        cur_lr: 5.000000000000001e-05
        entropy: 0.840692231853803
        entropy_coeff: 0.0
        kl: 0.017356043629588383
        policy_loss: -0.08558359454075495
        total_loss: 115.641189549764
        

In [ ]:
from rlskyjo.game.skyjo import SkyjoGame

env = PettingZooEnv(env_creator())
obs = env.reset()
done = {"__all__": False}
# run one iteration until done

for i in range(1000):
    if done["__all__"]:
        print("game done")
        break
    # get agent from current observation
    agent = list(obs.keys())[0]

    # format observation dict
    print(obs)
    obs = obs[agent]
    env.render()

    # get deterministic action
    # trainer.compute_single_action(obs, policy_id=agent)
    policy = trainer.get_policy(policy_id=agent)
    action_exploration_policy, _, action_info = policy.compute_single_action(obs)
    logits = action_info["action_dist_inputs"]
    action = logits.argmax()
    #
    print("agent ", agent, " action ", SkyjoGame.render_action_explainer(action))
    obs, reward, done, _ = env.step({agent: action})
    # observations contain original observations and the action mask
    # print(f"Obs: {obs}, Action: {action}, done: {done}")

env.render()
print(env.env.rewards)

/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")


{'player_0': {'observations': array([ 3, 10,  1,  0,  2,  0,  1,  1,  0,  1,  0,  0,  0,  1,  0,  0,  0,
        0, 15, 15,  9, 15, 15, 15, 15, 15, 15, 15, 15, 15,  2, 15,  9, 15,
       15, 15, 15, 15, 15, 15, 15, 15,  2, 15,  9, 15, 15, 15, 15, 15, 15,
       15, 15, 15,  2], dtype=int8), 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1], dtype=int8)}}
======= render board: ===== 
======= stats ============ 
next turn: draw by Player 0 
holding card player 0: empty 
discard pile top: 0 
======= Player 0 ========== 
[[u	 9	 u	 u]
 [u	 u	 u	 u]
 [u	 u	 u	 2]]
======= Player 1 ========== 
[[u	 u	 u	 3]
 [u	 u	 u	 u]
 [u	 u	 0	 u]]
======= Player 2 ========== 
[[u	 u	 u	 u]
 [u	 u	 u	 -2]
 [5	 u	 u	 u]]

agent  player_0  action  draw from drawpile
{'player_0': {'observations': array([ 3, 10,  1,  0,  2,  0,  1,  1,  0,  1,  0,  0,  0,  1,  0,  0,  0,
        0,  9, 15,  9, 15, 15, 15, 15, 15, 15, 15, 15, 15,  2, 15,  9, 15,
       

E0129 22:12:53.991486286   28715 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643490773.991444826","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643490773.991441540","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
E0129 22:41:44.046908525   14112 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643492504.046883176","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643492504.046880992","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
E0129 22:50:04.063877137   19203 backup_poller.cc:134]       Run client channel backup poller: {"created

In [ ]:
obs = {
    "observation": np.array(
        [
            15,
            15,
            7,
            4,
            15,
            -2,
            15,
            6,
            12,
            15,
            15,
            4,
            9,
            6,
            3,
            0,
            1,
            0,
            1,
            0,
            2,
            1,
            1,
            2,
            0,
            1,
            1,
            0,
            1,
            -2,
            4,
        ],
        dtype=np.int8,
    ),
    "action_mask": np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=np.int8),
}
action = trainer.compute_single_action(obs, policy_id=agent)

ValueError: ('Observation ({} dtype={}) outside given space ({})!', {'observation': array([15, 15,  7,  4, 15, -2, 15,  6, 12, 15, 15,  4,  9,  6,  3,  0,  1,
        0,  1,  0,  2,  1,  1,  2,  0,  1,  1,  0,  1, -2,  4], dtype=int8), 'action_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=int8)}, None, Dict(action_mask:Box([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], (26,), int8), observations:Box([-24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24
 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24 -24], [127 127 127 127 127 127 127 127 127 127 127 127 127 127 127 127 127 127
 127 127 127 127 127 127 127 127 127 127 127 127 127], (31,), int8)))

Try creating with rllib